In [ ]:
import os
import pandas as pd

from datasets import load_dataset, concatenate_datasets
from openai import AsyncOpenAI
import warnings


In [ ]:
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

client = AsyncOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY,  # 取得したAPIキーを入力してください
)


### データセットの準備

In [ ]:
jaquad_dataset = load_dataset("SkelterLabsInc/JaQuAD")

combined_data = concatenate_datasets(
    [jaquad_dataset["train"], jaquad_dataset["validation"]]
)

# 前処理：正解データの準備
combined_data_processed = []
for item in combined_data:
    answer_text = item["answers"]["text"][0] if item["answers"]["text"] else ""
    answer_start = (
        item["answers"]["answer_start"][0] if item["answers"]["answer_start"] else -1
    )
    combined_data_processed.append(
        {
            "question": item["question"],
            "context": item["context"],
            "answer_text": answer_text,
            "answer_start": answer_start,
        },
    )


# データが多いため10件に絞る
combined_data_processed = combined_data_processed[:10]
combined_data_processed[0]


{'question': '戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?',
 'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
 'answer_text': '手塚治虫',
 'answer_start': 0}

### llama3-70b-instructを用いて回答作成

In [ ]:
async def rag_generate_answer(client, question: str, context: str) -> str:
    """APIを使ってLLMによる解答を得る関数

    Question (str): 質問内容
        context (str): 解答の根拠になるテキスト情報

    Returns:
        response.choices[0].message.content(str): contextを元に回答したLLMの回答内容

    """
    response = await client.chat.completions.create(
        model="meta/llama3-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは優秀なアシスタントです。"
                "これから渡すコンテキストを元に問題に答えてください。"
                "答えられる文字数は少ないので単語のみなど簡潔な回答を心がけてください。",
            },
            {
                "role": "user",
                "content": f"Context: {context}. Answer the question:{question}",
            },
        ],
        max_tokens=15,
        temperature=0,
    )

    return response.choices[0].message.content


async def llm_generate_answer(client, question: str) -> str:
    """APIを使ってLLMによる解答を得る関数

    Question (str): 質問内容
        context (str): 解答の根拠になるテキスト情報

    Returns:
        response.choices[0].message.content(str): contextを元に回答したLLMの回答内容

    """
    response = await client.chat.completions.create(
        model="meta/llama3-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは優秀なアシスタントです。質問に答えてください。答えられる文字数は少ないので単語のみなど簡潔な回答を心がけてください。",
            },
            {
                "role": "user",
                "content": f"Answer the question: {question}",
            },
        ],
        max_tokens=15,
        temperature=0,
    )
    return response.choices[0].message.content


In [ ]:
for item in combined_data_processed:
    item["rag_answer_text"] = await rag_generate_answer(
        client, item["question"], item["context"]
    )
    item["llm_answer_text"] = await llm_generate_answer(client, item["question"])


### nemotron-4-340b-rewardで評価

In [ ]:
def get_scores_from_response(openai_response_template):
    """OpenAI APIのレスポンスからトークンごとのスコアを抽出する関数

    Args:
        openai_response_template: OpenAI APIのレスポンスオブジェクト

    Returns:
        dict: トークンとそのログ確率を含む辞書
    """
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict


async def get_response_and_scores(client, question, response_content):
    """指定された質問と応答内容をもとにスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        question (str): 質問文
        response_content (str): 応答内容

    Returns:
        dict: トークンごとのスコアを含む辞書
    """
    messages = [
        {"role": "user", "content": question},
        {"role": "assistant", "content": response_content},
    ]
    response = await client.chat.completions.create(
        model="nvidia/nemotron-4-340b-reward",
        messages=messages,
    )
    scores = get_scores_from_response(response)
    return scores


async def rag_process_question_response_pairs(client, combined_data_processed):
    """質問と応答のペアに対してスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        combined_data_processed (list): 質問と応答のペアを含むデータリスト

    Returns:
        list: 各質問と応答ペアごとのスコア情報を含むリスト
    """
    results = []
    for i in range(len(combined_data_processed)):
        question = combined_data_processed[i]["question"]
        answer = combined_data_processed[i]["rag_answer_text"]

        scores = await get_response_and_scores(
            client,
            question,
            answer,
        )
        results.append((question, answer, scores))
    return results


async def llm_process_question_response_pairs(client, combined_data_processed):
    """質問と応答のペアに対してスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        combined_data_processed (list): 質問と応答のペアを含むデータリスト

    Returns:
        list: 各質問と応答ペアごとのスコア情報を含むリスト
    """
    results = []
    for i in range(len(combined_data_processed)):
        question = combined_data_processed[i]["question"]
        answer = combined_data_processed[i]["llm_answer_text"]

        scores = await get_response_and_scores(
            client,
            question,
            answer,
        )
        results.append((question, answer, scores))
    return results


In [23]:
llm_results = await llm_process_question_response_pairs(client, combined_data_processed)
rag_results = await rag_process_question_response_pairs(client, combined_data_processed)


In [24]:
llm_results_df = pd.DataFrame(
    [
        (
            question,
            answer,
            scores["helpfulness"],
            scores["correctness"],
            scores["coherence"],
            scores["complexity"],
            scores["verbosity"],
        )
        for question, answer, scores in llm_results
    ],
    columns=[
        "question",
        "llm_answer",
        "helpfulness",
        "correctness",
        "coherence",
        "complexity",
        "verbosity",
    ],
)
llm_results_df


,question,llm_answer,helpfulness,correctness,coherence,complexity,verbosity
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,手塚治虫,3.109375,3.171875,3.687500,0.792969,0.691406
1,手塚治虫の出身地はどこになりますか?,兵庫県宝塚市,2.796875,2.875000,3.765625,0.722656,0.726562
2,手塚治虫の漫画家としてのデビュー作は何かな?,マアチャンの宇宙旅行,2.265625,2.468750,3.390625,0.707031,0.855469
3,1947年に大阪に赤本ブームが起こったのはなぜ?,GHQの検閲緩和。,1.187500,1.234375,2.906250,0.554688,0.304688
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,「鉄腕アトム」,2.578125,2.906250,3.515625,0.761719,0.687500
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,アドルフに告ぐ,3.328125,3.796875,3.828125,0.886719,0.753906
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,芦屋中学。,1.062500,1.101562,3.234375,0.359375,0.609375
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,戦争体験。,0.574219,0.925781,3.109375,0.492188,0.373047
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,『どついたれ』,0.228516,0.539062,3.062500,0.343750,0.137695
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,1996-2003,0.283203,0.253906,3.046875,0.211914,0.195312


In [25]:
rag_results_df = pd.DataFrame(
    [
        (
            question,
            answer,
            scores["helpfulness"],
            scores["correctness"],
            scores["coherence"],
            scores["complexity"],
            scores["verbosity"],
        )
        for question, answer, scores in rag_results
    ],
    columns=[
        "question",
        "rag_answer",
        "helpfulness",
        "correctness",
        "coherence",
        "complexity",
        "verbosity",
    ],
)
rag_results_df


,question,rag_answer,helpfulness,correctness,coherence,complexity,verbosity
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,手塚治虫,3.109375,3.171875,3.687500,0.792969,0.691406
1,手塚治虫の出身地はどこになりますか?,大阪府豊能郡豊中町（現在の豊中市）,3.015625,2.984375,3.687500,0.757812,0.820312
2,手塚治虫の漫画家としてのデビュー作は何かな?,マアチャンの日記帳,2.312500,2.437500,3.406250,0.566406,0.707031
3,1947年に大阪に赤本ブームが起こったのはなぜ?,酒井七馬原案の描き下ろし単行本,0.279297,0.087891,2.375000,0.049316,0.761719
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,鉄腕アトム,2.500000,2.906250,3.515625,0.742188,0.746094
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,アドルフに告ぐ,3.328125,3.796875,3.828125,0.886719,0.753906
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,大阪府立北野中学校,1.390625,1.515625,3.265625,0.488281,0.777344
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,大阪大空襲。,0.707031,0.957031,3.093750,0.486328,0.369141
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,『紙の砦』(1974年),0.886719,1.085938,3.203125,0.570312,0.380859
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,1946年1月1日-3月31日,2.265625,2.343750,3.531250,0.621094,0.632812


In [26]:
rag_results_df["answer"] = ""
for i in range(len(rag_results_df)):
    rag_results_df["answer"][i] = combined_data_processed[i]["answer_text"]
rag_results_df


C:\Users\sige1\AppData\Local\Temp\ipykernel_30956\3491040805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rag_results_df["answer"][i] = combined_data_processed[i]["answer_text"]
C:\Users\sige1\AppData\Local\Temp\ipykernel_30956\3491040805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rag_results_df["answer"][i] = combined_data_processed[i]["answer_text"]
C:\Users\sige1\AppData\Local\Temp\ipykernel_30956\3491040805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,question,rag_answer,helpfulness,correctness,coherence,complexity,verbosity,answer
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,手塚治虫,3.109375,3.171875,3.687500,0.792969,0.691406,手塚治虫
1,手塚治虫の出身地はどこになりますか?,大阪府豊能郡豊中町（現在の豊中市）,3.015625,2.984375,3.687500,0.757812,0.820312,兵庫県宝塚市
2,手塚治虫の漫画家としてのデビュー作は何かな?,マアチャンの日記帳,2.312500,2.437500,3.406250,0.566406,0.707031,『マアチャンの日記帳』
3,1947年に大阪に赤本ブームが起こったのはなぜ?,酒井七馬原案の描き下ろし単行本,0.279297,0.087891,2.375000,0.049316,0.761719,『新寶島』がベストセラーとなり
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,鉄腕アトム,2.500000,2.906250,3.515625,0.742188,0.746094,『鉄腕アトム』
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,アドルフに告ぐ,3.328125,3.796875,3.828125,0.886719,0.753906,『アドルフに告ぐ』
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,大阪府立北野中学校,1.390625,1.515625,3.265625,0.488281,0.777344,大阪府立北野中学校
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,大阪大空襲。,0.707031,0.957031,3.093750,0.486328,0.369141,大阪大空襲
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,『紙の砦』(1974年),0.886719,1.085938,3.203125,0.570312,0.380859,『紙の砦』
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,1946年1月1日-3月31日,2.265625,2.343750,3.531250,0.621094,0.632812,1946年1月1日-3月31日


In [ ]:
rag_results_df.columns = [""]

{'question': '戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?',
 'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
 'answer_text': '手塚治虫',
 'answer_start': 0,
 'rag_answer_text': '松本零士'}